In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [3]:
all_sites5 = all_sites[12400:]

In [4]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [8]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1500
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [9]:
def apply_sentiment_model(df):
    
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
        
    finally:
        df.to_csv('../data/labeled_texts5.csv', index=False)
    
    return df

In [10]:
apply_sentiment_model(all_sites5)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,site,url_link,author,game,company,genre,platform,text,score,stars
12400,3D Juegos,https://www.3djuegos.com/juegos/analisis/12089...,Alberto Pastor,Legend of Grimrock - Análisis,None,None,PC Mac Linux,"Lo clásico está de moda, y Legend of Grimrock ...",8.4,"[{'label': '4 stars', 'score': 0.4281089603900..."
12401,Vandal,https://vandal.elespanol.com/analisis/xbone/ha...,Carlos Leiva,Halo 5: Guardians - Análisis,Microsoft,Shooter en primera persona,Xbox One,Si hay una saga de videojuegos capaz de vender...,9.2,"[{'label': '3 stars', 'score': 0.4472388923168..."
12402,Gamereactor,https://www.gamereactor.es/hades-analisis/?sid...,Ricardo C. Esteves,Hades,None,Acción,"Nintendo Switch, PC",El periplo por un limbo llamado acceso anticip...,8.0,"[{'label': '4 stars', 'score': 0.5017783641815..."
12403,meristation,https://as.com/meristation/2010/01/15/analisis...,Joaquín Relaño,Resident Evil Archives: Resident Evil Zero,Capcom,Acción,Wii,La historia del desarrollo de Resident Evil Ze...,6.0,"[{'label': '4 stars', 'score': 0.4219374954700..."
12404,meristation,https://as.com/meristation/2009/08/14/analisis...,None,Killing Floor,None,"Acción, First Person Shooter",PC,Tú y tu equipo tenéis una misión: exterminar a...,7.0,"[{'label': '4 stars', 'score': 0.2990265190601..."
...,...,...,...,...,...,...,...,...,...,...
15546,meristation,https://as.com/meristation/2009/02/23/analisis...,Pablo González,Eledees: The Adventures of Kai and Zero,None,Puzzle,DS,Konami fue una compañía bastante inteligente c...,7.5,"[{'label': '3 stars', 'score': 0.3707822263240..."
15547,Vandal,https://vandal.elespanol.com/analisis/switch/t...,Sergio Martín,The Stretchers - Análisis,Nintendo,Puzle,Switch,"El equipo de Tarsier Studios, responsables ent...",7.7,"[{'label': '3 stars', 'score': 0.3597940206527..."
15548,3D Juegos,https://www.3djuegos.com/juegos/analisis/3486/...,Jesús Bella,Deadly Creatures - Análisis,None,None,Wii,Si estás harto de ponerte siempre del lado de ...,8.0,"[{'label': '3 stars', 'score': 0.4155605137348..."
15549,3D Juegos,https://www.3djuegos.com/juegos/analisis/11213...,Sergio Martín,Dragon Ball Z Ultimate Tenkaichi - Análisis,None,None,PS3 X360,Namco Bandai y Spike vuelven a la carga con la...,7.0,"[{'label': '4 stars', 'score': 0.4572119116783..."


In [11]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [12]:
all_sites5['stars_mean'] = all_sites5['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [14]:
all_sites5['score_label'] = all_sites5['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
def score_half(score):
    return score / 2

In [16]:
all_sites5['score_adj'] = all_sites5['score'].apply(lambda x: score_half(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
all_sites5.to_csv('../data/scored_texts5.csv', index=False)

In [18]:
all_sites5

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label,score_adj
12400,3D Juegos,https://www.3djuegos.com/juegos/analisis/12089...,Alberto Pastor,Legend of Grimrock - Análisis,None,None,PC Mac Linux,"Lo clásico está de moda, y Legend of Grimrock ...",8.4,"[{'label': '4 stars', 'score': 0.4281089603900...",3.43,'4 starts',4.20
12401,Vandal,https://vandal.elespanol.com/analisis/xbone/ha...,Carlos Leiva,Halo 5: Guardians - Análisis,Microsoft,Shooter en primera persona,Xbox One,Si hay una saga de videojuegos capaz de vender...,9.2,"[{'label': '3 stars', 'score': 0.4472388923168...",3.33,'5 starts',4.60
12402,Gamereactor,https://www.gamereactor.es/hades-analisis/?sid...,Ricardo C. Esteves,Hades,None,Acción,"Nintendo Switch, PC",El periplo por un limbo llamado acceso anticip...,8.0,"[{'label': '4 stars', 'score': 0.5017783641815...",3.50,'4 starts',4.00
12403,meristation,https://as.com/meristation/2010/01/15/analisis...,Joaquín Relaño,Resident Evil Archives: Resident Evil Zero,Capcom,Acción,Wii,La historia del desarrollo de Resident Evil Ze...,6.0,"[{'label': '4 stars', 'score': 0.4219374954700...",3.75,'3 starts',3.00
12404,meristation,https://as.com/meristation/2009/08/14/analisis...,None,Killing Floor,None,"Acción, First Person Shooter",PC,Tú y tu equipo tenéis una misión: exterminar a...,7.0,"[{'label': '4 stars', 'score': 0.2990265190601...",3.71,'4 starts',3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15546,meristation,https://as.com/meristation/2009/02/23/analisis...,Pablo González,Eledees: The Adventures of Kai and Zero,None,Puzzle,DS,Konami fue una compañía bastante inteligente c...,7.5,"[{'label': '3 stars', 'score': 0.3707822263240...",3.57,'4 starts',3.75
15547,Vandal,https://vandal.elespanol.com/analisis/switch/t...,Sergio Martín,The Stretchers - Análisis,Nintendo,Puzle,Switch,"El equipo de Tarsier Studios, responsables ent...",7.7,"[{'label': '3 stars', 'score': 0.3597940206527...",3.25,'4 starts',3.85
15548,3D Juegos,https://www.3djuegos.com/juegos/analisis/3486/...,Jesús Bella,Deadly Creatures - Análisis,None,None,Wii,Si estás harto de ponerte siempre del lado de ...,8.0,"[{'label': '3 stars', 'score': 0.4155605137348...",3.50,'4 starts',4.00
15549,3D Juegos,https://www.3djuegos.com/juegos/analisis/11213...,Sergio Martín,Dragon Ball Z Ultimate Tenkaichi - Análisis,None,None,PS3 X360,Namco Bandai y Spike vuelven a la carga con la...,7.0,"[{'label': '4 stars', 'score': 0.4572119116783...",4.00,'4 starts',3.50
